In [1]:
!pip install -r requirements.txt
!pip install transformers[deepspeed]
!pip install mpi4py

  Cloning https://github.com/huggingface/transformers.git (to revision 204737fcc5213be86dc9b83f6db42bc7aefb574b) to /tmp/pip-install-cy1sm_07/transformers_b50677dc0652409bbc3a1011775870ac
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-cy1sm_07/transformers_b50677dc0652409bbc3a1011775870ac
  Running command git rev-parse -q --verify 'sha^204737fcc5213be86dc9b83f6db42bc7aefb574b'
  Running command git fetch -q https://github.com/huggingface/transformers.git 204737fcc5213be86dc9b83f6db42bc7aefb574b
  Running command git checkout -q 204737fcc5213be86dc9b83f6db42bc7aefb574b
  Resolved https://github.com/huggingface/transformers.git to commit 204737fcc5213be86dc9b83f6db42bc7aefb574b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

# Fine-tuning

In [2]:
import os
import json
import pandas as pd


raw_data = pd.read_csv("ProblemSolutionPythonV3.csv")
raw_data.head()

,Unnamed: 0,Problem,Python Code
0,0,Write a NumPy program to repeat elements of an...,"import numpy as np\rx = np.repeat(3, 4)\rprint..."
1,1,Write a Python function to create and print a ...,def printValues():\n\tl = list()\n\tfor i in r...
2,2,Write a Python program to remove duplicates fr...,"import itertools\rnum = [[10, 20], [40], [30, ..."
3,3,Write a NumPy program to compute the x and y c...,import numpy as np\rimport matplotlib.pyplot a...
4,4,Write a Python program to alter a given SQLite...,import sqlite3\rfrom sqlite3 import Error\rdef...


In [3]:
raw_data = raw_data.dropna()
data_path = "CodeT5/data/concode/"
data = []
for _, line in raw_data.iterrows():
  data.append({
      "code": line["Python Code"], 
      "nl": line["Problem"]
  })

train_size = int(len(data) * 0.9)
with open(os.path.join(data_path, "train.json"), "w") as fp:
  for ind in range(0, train_size):
    fp.write(json.dumps(data[ind]))
    fp.write("\n")

with open(os.path.join(data_path, "dev.json"), "w") as fp:
  for ind in range(train_size, len(data)):
    fp.write(json.dumps(data[ind]))
    fp.write("\n")

with open(os.path.join(data_path, "test.json"), "w") as fp:
  for ind in range(train_size, len(data)):
    fp.write(json.dumps(data[ind]))
    fp.write("\n")

Hyperparameters chosen for fine-tuning:
- batch size = 8

In [4]:
# checkpoint is saved at ./saved_models/concode/codet5_base_all_lr10_bs8_src320_trg150_pat3_e30/checkpoint-best-bleu/pytorch_model.bin
!python CodeT5/sh/run_exp.py --task concode --sub_task none --model_tag codet5_base --do_train true --checkpoint_path ./saved_models/concode/codet5_base_all_lr10_bs8_src320_trg150_pat3_e30/checkpoint-best-bleu/pytorch_model.bin

============================Start Running==========================
bash CodeT5/sh/exp_with_args.sh concode none codet5_base 0 -1 8 10 320 150 3 30 1000 saved_models tensorboard results/concode_codet5_base.txt true ./saved_models/concode/codet5_base_all_lr10_bs8_src320_trg150_pat3_e30/checkpoint-best-bleu/pytorch_model.bin None


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.6/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 116
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbit

# Inference

Instructions: load two checkpoints from here https://drive.google.com/drive/folders/1Z9WmhoMdgk-_DRY5QMhVhX_EtMJH2RvU?usp=share_link

Create folder to unzip checkpoints there

Run inference for the initial CodeT5 model for java. Codebleu = 12.8152

In [44]:
!python CodeT5/sh/run_exp.py --task concode --sub_task none \
    --model_tag codet5_base --do_train false

============================Start Running==========================
bash CodeT5/sh/exp_with_args.sh concode none codet5_base 0 -1 8 10 320 150 3 30 1000 saved_models tensorboard results/concode_codet5_base.txt false None None


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.6/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 116
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda116_nocublaslt.so...
03/27/2023 20:07:36 - INFO - __main__ -   Namespace(task='concode', s

Run inference for fine-tuned model. Pass path of the checkpoint here. Resu;ts should be bleu = 20.68 and codebleu = 29

In [53]:
!python CodeT5/sh/run_exp.py --task concode_nl2py --sub_task none \
    --model_tag codet5_base --do_train false \
    --checkpoint_path ./saved_models/concode/codet5_base_all_lr10_bs8_src320_trg150_pat3_e30/checkpoint-best-bleu/pytorch_model.bin 

============================Start Running==========================
bash CodeT5/sh/exp_with_args.sh concode_nl2py none codet5_base 0 -1 8 10 320 150 3 30 1000 saved_models tensorboard results/concode_nl2py_codet5_base.txt false ./saved_models/concode/codet5_base_all_lr10_bs8_src320_trg150_pat3_e30/checkpoint-best-bleu/pytorch_model.bin None


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.6/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 116
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsan

## 8-bit

Now test zero degradation matrix multiplication for Large Language Models described here: https://huggingface.co/blog/hf-bitsandbytes-integration. 

In [ ]:
!python CodeT5/sh/run_exp.py --task concode --sub_task none \
    --model_tag codet5_base --do_train false \
    --compress 8bit

============================Start Running==========================
bash CodeT5/sh/exp_with_args.sh concode none codet5_base 0 -1 8 10 320 150 3 30 1000 saved_models tensorboard results/concode_codet5_base.txt false None 8bit


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.6/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 116
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda116_nocublaslt.so...
03/27/2023 21:00:47 - INFO - __main__ -   Namespace(task='concode', s

## Distillation

Do not run 2 cells below. It was used to train a student network

In [15]:
import os
import json


def restructure_data(src_dir, task):
  dst_dir = os.path.join(src_dir, f"{task}_distillation")
  os.mkdir(dst_dir)

  for file in ['train', 'dev', 'test']:
    out_file = file if file != 'dev' else 'val'
    with open(os.path.join(src_dir, task, file + ".json")) as fp1, open(os.path.join(dst_dir, out_file + ".source"), 'w') as fp2, open(os.path.join(dst_dir, out_file + ".target"), 'w') as fp3:
      for line in fp1:
        data = json.loads(line)
        fp2.write(data['nl'] + '\n')
        fp3.write(data['code'] + '\n')

/notebooks/saved_models/concode/codet5_base_all_lr10_bs8_src320_trg150_pat3_e30/checkpoint-best-bleu/
restructure_data('./CodeT5/data', 'concode')

!python distillation.py \
  --teacher Salesforce/codet5-base --data_dir data/concode_distillation \
  --tokenizer_name Salesforce/codet5-base \
  --student_decoder_layers 6 --student_encoder_layers 12 \
  --freeze_encoder --freeze_embeds \
  --learning_rate=3e-4 \
  --do_train \
  --do_predict \
  --fp16 --fp16_opt_level=O1 \
  --val_check_interval 0.1 --n_val 1000 --eval_beams 2 --length_penalty=0.5 \
  --max_target_length=60 --val_max_target_length=60 --test_max_target_length=100 \
  --model_name_or_path IGNORED \
  --alpha_hid=3. \
  --train_batch_size=8 --eval_batch_size=8 --gradient_accumulation_steps=2 \
  --sortish_sampler \
  --num_train_epochs=6 \
  --warmup_steps 500 \
  --output_dir distilt5_12_6 \
  --task translation \
  --overwrite_output_dir \
  "$@"

Now insert path to student model here and run cell to see example

In [33]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration


# specify path to student model here
PATH_TO_STUDENT = '/content/LMInference/transformers/examples/research_projects/seq2seq-distillation/distilt5_12_6/student'

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')
model_student = T5ForConditionalGeneration.from_pretrained(PATH_TO_STUDENT)

text = "def greet(user): print(f'hello <extra_id_0>!')"
input_ids = tokenizer(text, return_tensors="pt").input_ids

# simply generate one code span
generated_ids = model.generate(input_ids)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))
generated_ids_student = model_student.generate(input_ids)
print(tokenizer.decode(generated_ids_student[0], skip_special_tokens=True))

Traceback (most recent call last):
  File "/notebooks/transformers/examples/research_projects/seq2seq-distillation/distillation.py", line 12, in <module>
    from finetune import SummarizationModule, TranslationModule
  File "/notebooks/transformers/examples/research_projects/seq2seq-distillation/finetune.py", line 16, in <module>
    from callbacks import Seq2SeqLoggingCallback, get_checkpoint_callback, get_early_stopping_callback
  File "/notebooks/transformers/examples/research_projects/seq2seq-distillation/callbacks.py", line 10, in <module>
    from utils import save_json
  File "/notebooks/transformers/examples/research_projects/seq2seq-distillation/utils.py", line 17, in <module>
    from sacrebleu import corpus_bleu
  File "/usr/local/lib/python3.9/dist-packages/sacrebleu/__init__.py", line 21, in <module>
    from .utils import smart_open, SACREBLEU_DIR, download_test_set  # noqa: F401
  File "/usr/local/lib/python3.9/dist-packages/sacrebleu/utils.py", line 597, in <module>
  

Run next cell to compare memory occupied by models. Results should be 891528192 for main model and 664980480 for student model

Test BLEU for initial model is 8.2807

In [ ]:
print(model.get_memory_footprint())
print(model_student.get_memory_footprint())